<a href="https://colab.research.google.com/github/TJSun009/University-Projects/blob/main/Test_Categorisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble
This model is designed to look at categorising unit tests in order to perform downstream tasks (such as mapping test types to certain actions)

List of sources used:


*   [https://www.geeksforgeeks.org/unit-testing-software-testing](https://www.geeksforgeeks.org/unit-testing-software-testing)
*   [https://www.javatpoint.com/unit-testing](https://www.javatpoint.com/unit-testing)
*   [https://www.imperva.com/learn/application-security/black-box-testing](https://www.imperva.com/learn/application-security/black-box-testing)
*   [https://www.imperva.com/learn/application-security/white-box-testing/](https://www.imperva.com/learn/application-security/white-box-testing/)



Broadly speaking documentation  refers to three broad unit test types:


* Black Box Testing - Testing a system with no knowledge of its internals
    * This involves testing the user interface i.e input and outputs
    * **Utility** - checks that the *system as a whole is working* as expected
    * Example: 
        * checking that it is possible to log in using correct user credentials, and not possible to log in using wrong credentials.
    * Includes checking:
        * input/output **formats** (length and REGEX)
        * **boundary** values
* White Box Testing - Testing a system with knowledge of internals (source code, documentation etc.)
    * This involves testing behaviour of the system from developer perspective
    * **Utility** - can uncover structural problems, hidden errors and problems with specific components; ensures code is comprehensively covered
    * Example:
        * 
    * Includes checking:
        * **security vulnerabilities**
        * **loop testing**
        * **types**
        * **data flow**
        * **control flow** - order of execution
    * Code coverage Techniques:
        * **branch coverage**
        * **statement coverage**
        * **path coverage** - looking at executed code paths and their relevance
* Gray Box Testing - Testing with partial knowledge of the system's internals
    * It's a combination of White and Gray Box Testing




# Assumptions
Given that we will be using the source code we can assume our tests will either be White or Grey Box

The most common tests expected to come up are:

* Bounds Testing
* Branching Statements
* Loop Testing
* Format
* Error

The basic things that will be checked are:
* Return type
* Return length
* Function calls
* Exception raised



# Test 1

We'll work with the pyUnittest_Tutorial which provides source code and tests for a simple calculator

Our work assumes both the code and tests are correct and functioning

## Start with two generic labels

In [ ]:
labels = ["valid", "invalid"]

## Import Test File

In [ ]:
!git clone https://github.com/Teatoller/pyUnittest_Tutorial.git

Cloning into 'pyUnittest_Tutorial'...
remote: Enumerating objects: 27, done.
remote: Total 27 (delta 0), reused 0 (delta 0), pack-reused 27
Unpacking objects: 100% (27/27), done.


## Import libraries

### Graph generator Setup

In [ ]:
#@title Git Config
import getpass
!git config --global user.email "emmanuelo009.oe@gmail.com"
!git config --global user.name "TJSun009"

TARGET_DIR = "/content/typilus"
BRANCH_NAME = "time.clock-bugfix"
ACCESS_TOKEN = getpass.getpass()
REMOTE = f"https://{ACCESS_TOKEN}@github.com/TJSun009/typilus.git"

··········


In [ ]:
#@title Typilus Forked Bugfix Branch

!git clone https://github.com/TJSun009/typilus.git

!(cd {TARGET_DIR}/; \
git pull {REMOTE} {BRANCH_NAME}; \
git checkout {BRANCH_NAME})

Cloning into 'typilus'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 161 (delta 41), reused 118 (delta 20), pack-reused 0
Receiving objects: 100% (161/161), 162.54 KiB | 3.19 MiB/s, done.
Resolving deltas: 100% (41/41), done.
From https://github.com/TJSun009/typilus
 * branch            time.clock-bugfix -> FETCH_HEAD
Updating 69c377b..1467053
Fast-forward
 src/data_preparation/scripts/graph_generator/extract_graphs.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
Branch 'time.clock-bugfix' set up to track remote branch 'time.clock-bugfix' from 'origin'.
Switched to a new branch 'time.clock-bugfix'


In [ ]:
#@title Add to Path
import sys

GRAPH_GENERATOR_PATH = "/content/typilus/src/data_preparation/scripts/graph_generator"

#change graphgenerator to module
!touch {GRAPH_GENERATOR_PATH}/__init__.py

SRC_FOLDER = "/content/pyUnittest_Tutorial/"
TYPING_RULES_PATH = "/content/typilus/src/data_preparation/metadata/typingRules.json"
CORPUS_DUPLICATES = "/content/drive/MyDrive/Year 3/Dissertation/Projects/corpus_duplicates.json"

if (GRAPH_GENERATOR_PATH not in sys.path):
  sys.path.append(f"{GRAPH_GENERATOR_PATH}")

### Graph generator Functions

In [ ]:
!pip install -Uqqq dpu_utils typed_ast
import extract_graphs, os, gzip, json


def generate_graphs(src, dest = 0):
  dest = src if dest == 0 else dest
  extract_graphs.main({
      "SOURCE_FOLDER": src,
      "SAVE_FOLDER": dest,
      "TYPING_RULES": TYPING_RULES_PATH,
      "DUPLICATES_JSON": CORPUS_DUPLICATES
  })

def graphToJson(file):
  json_content = []
  with gzip.open(file) as f:
    for line in f:
      line = line.rstrip()
      if line:
        obj = json.loads(line)
        json_content.append(obj)
  return json_content

     |████████████████████████████████| 73 kB 1.5 MB/s 
     |████████████████████████████████| 897 kB 9.7 MB/s 
     |████████████████████████████████| 135 kB 71.4 MB/s 
     |████████████████████████████████| 383 kB 64.8 MB/s 
     |████████████████████████████████| 1.3 MB 65.3 MB/s 
     |████████████████████████████████| 4.0 MB 67.2 MB/s 
     |████████████████████████████████| 172 kB 74.6 MB/s 
     |████████████████████████████████| 90 kB 9.1 MB/s 
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 41 kB 744 kB/s 


In [ ]:
SRC = "/content/pyUnittest_Tutorial"

generate_graphs(SRC)

Exploring folders ...
/content/pyUnittest_Tutorial/app/ __init__.py
/content/pyUnittest_Tutorial/app/calculator.py
/content/pyUnittest_Tutorial/test/__init__.py
/content/pyUnittest_Tutorial/test/test_calculator.py
Building and saving the type graph...
Building type graph for project... (0 elements to process)
Done building type graph
Done.
Generated 4 graphs out of 4 snippets

Execution in:  0.09160692000000026  seconds


In [ ]:
content = graphToJson("/content/pyUnittest_Tutorial/all-graphs000.jsonl.gz")

## Split Test by Function



*   Imports and setUp files will be needed in the 



In [ ]:
import ast, astunparse, pathlib
from re import sub
# for visual debug
# !pip install -Uqqq pprintast
# from pprintast import pprintast as ppast

def flatten(ls):
  return [item for l in ls for item in l]


def title_case(s):
  s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].upper(), s[1:]])

def split_tests(src_folder, dest_folder = "./"):
  src_folder = pathlib.Path(src_folder)

  matches = [src_folder.glob("**/test_*.py"), src_folder.glob("**/*_test.py")]
  matches = flatten(matches)
  
  imports = []
  functions = {}
  
  for file in matches:
    
    with open(file, 'r') as f:
      code = f.read()
      head = ast.parse(code)
      for node in ast.walk(head):
        try:
          for x in node.body:
            if (isinstance(x, ast.Import) or isinstance(x, ast.ImportFrom)):
              imports.append(x)
            elif isinstance(x, ast.FunctionDef):
              functions[x.name] = x
        except:
          pass

  count = 0

  for _,func in functions.items():
    # ignore setup file
    if(_ != "setUp"):
      test_file_head = ast.Module(body = [])
      # add imports
      for i in imports:
        test_file_head.body.append(i)
      
      className = title_case(file.name.replace(".py", ''))

      # add class definition and setup and test functions
      test_file_head.body.append(
          ast.ClassDef(
              f"{className}{count}",
              [ast.Name(id='unittest.TestCase', ctx=ast.Load())],
              keywords = [],# add 
              body = [
                  functions["setUp"],
                  func
                  ],
              decorator_list = []
          )
      )

      # create a folder where the files can go
      dest_path = pathlib.Path(dest_folder, file.name.replace(".py", ''))
      dest_path.mkdir(parents = True, exist_ok = True) 

      # files are distinguished by their parent folder and each function by a counter
      with open(pathlib.Path(dest_path, f"{count}.py"), 'w') as f:
        code = astunparse.unparse(test_file_head)
        f.write(code)

      count+=1

## Do split tests

In [ ]:
SPLIT_TEST_DEST = "/content/split_tests/"

In [ ]:
split_tests(SRC, SPLIT_TEST_DEST)

## Generate Graphs

In [ ]:
generate_graphs("/content/split_tests/test_calculator")

Exploring folders ...
/content/split_tests/test_calculator/1.py
/content/split_tests/test_calculator/3.py
/content/split_tests/test_calculator/2.py
/content/split_tests/test_calculator/0.py
Building and saving the type graph...
Building type graph for project... (0 elements to process)
Done building type graph
Done.
Generated 4 graphs out of 4 snippets

Execution in:  0.03364505399999995  seconds


In [ ]:
graphToJson("/content/split_tests/test_calculator/all-graphs000.jsonl.gz")
# graphToJson("/content/split_tests/test_calculator/_type_lattice.json.gz")
# print(example)
# print(f"nodes length = {len(set(example[0]['nodes']))}, token lengths = {len(example[0]['token-sequence'])}")

[{'nodes': ['Module',
   'Import',
   'alias',
   'ImportFrom',
   'ClassDef',
   'class',
   'TestCalculator1',
   '(',
   'Attribute',
   'Name',
   'unittest',
   'unittest',
   '.',
   'TestCase',
   'unittest.TestCase',
   ')',
   '<INDENT>',
   'FunctionDef',
   'def',
   'setUp',
   'setUp',
   '(',
   'arguments',
   'arg',
   'self',
   'self',
   ',',
   ')',
   '<INDENT>',
   'Assign',
   'Name',
   'self',
   '.',
   'calc',
   'Attribute',
   'self.calc',
   'Call',
   '=',
   'Name',
   'Calculator',
   'Calculator',
   '(',
   ')',
   '<DEDENT>',
   '<NL>',
   'FunctionDef',
   'def',
   'test_calculator_returns_error_message_if_both_args_not_numbers',
   'test_calculator_returns_error_message_if_both_args_not_numbers',
   '(',
   'arguments',
   'arg',
   'self',
   'self',
   ',',
   ')',
   '<INDENT>',
   'Expr',
   'Call',
   'Attribute',
   'Name',
   'self',
   '.',
   'assertRaises',
   'self.assertRaises',
   '(',
   'Name',
   'ValueError',
   'ValueError',
   '

## Feed Data to Graph Network

In [ ]:
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

### Helper Functions

In [ ]:
## tokenize edges
## edges have features of CHILD, NEXT, LAST_LEXICAL_USE, NEXT_USE, COMPUTED_FROM, OCCURRENCE_OF, SUBTOKEN_OF
## could try different methods to aggregate edge features
## each feature would correspond to an index

from keras.preprocessing.text import Tokenizer

def edgesToFeatureVector(embedding, aggregation = "sum"):
  no_nodes = len(embedding["nodes"])
  edges = embedding["edges"]
  feature_vect = []
  
  # loop through all edge_types and add them, use index to refer to type
  for type_idx, edge_type in enumerate(list(edges.keys())):
    for node in edges[edge_type].keys():
      for val in edges[edge_type][node]:
        # feature_vect.append([int(node), val, type_idx])
        feature_vect.append([int(node), val])

  
  return np.array(feature_vect)

# get the label for the method using the methd name
def getLabel(embedding):
    function_name = embedding["supernodes"]["48"]["name"]

    return labels[1] if (function_name.find("error")) else labels[0]

# get embedding of node texts
def getWordVectors(embedding, technique="one-hot"):
  if(technique == "one-hot"):
    words = []
    for graph in graphs:
      [words.append(token) for token in graph["nodes"]]

    
    max_nodes = max([len(node) for node in (embedding["nodes"] for embedding in graphs)])

    words = list(set(words))

    word_vects = []

    for token in embedding["nodes"]:
      word_vect = np.zeros((1, len(words)), dtype=object)
      word_vect[0][words.index(token)] = 1
      word_vects.append(word_vect)

    while len(word_vects) < max_nodes:
      word_vects.append(np.zeros((1, len(words)), dtype=object))
    
    return np.concatenate(np.array(word_vects, dtype=object))

# Extract node_features, edges and weights from a given graph
def getGraphInfo(graph):
  # Create an edges array (sparse adjacency matrix) of shape [3, num_edges].
  edges = np.array(edgesToFeatureVector(graph)).T

  # Create an edge weights array of ones.
  edge_weights = tf.ones(shape=edges.shape[1], dtype=tf.float32)

  # Create a node features array of shape [num_nodes, num_features].
  node_features = getWordVectors(graph)

  # Create graph info dictionary with node_features, edges, and edge_weights.
  return {"node_features": node_features, "edges": edges, "edge_weights": edge_weights}
  # return {"node_features": node_features, "edges": edges, "edge_weights": None}

In [ ]:
def transformGraphForGNN(graph):
    return {
        "name": graph["supernodes"]["48"]["name"],
        "info": getGraphInfo(graph),
        "label": getLabel(graph)
    }

def groupByLabel(data, labels):
  groups = [[] for i in labels]

  for name in data.keys():
    graph = data[name]
    which = labels.index(graph["label"])
    groups[which].append(graph)
  
  return [np.array(group) for group in groups]

In [ ]:
graphs = graphToJson("/content/split_tests/test_calculator/all-graphs000.jsonl.gz")

### Prep Datasets


*   Test and Train Features



In [ ]:
num_classes = len(labels)

## Create Feed Forward Network

In [ ]:
def create_ffn(hidden_units, dropout_rate, name=None):
    fnn_layers = []

    for units in hidden_units:
        # Batch Normalization normalizes unit values in batches (using its mean and standard dev) to create even spread 
        fnn_layers.append(layers.BatchNormalization())
        # Dropout removes units randomly to prevent overfitting
        fnn_layers.append(layers.Dropout(dropout_rate))
        # Dense layer every node conected to the others
        fnn_layers.append(layers.Dense(units, activation=tf.nn.gelu))

    return keras.Sequential(fnn_layers, name=name)

## Create Train and Evaluation Experiment

In [ ]:
# Train and Arch variables
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5
num_epochs = 300
batch_size = 256

In [ ]:
def run_experiment(model, x_train, y_train):
    # Compile the model.
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )
    # Create an early stopping callback.
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_split=0.15,
        callbacks=[early_stopping],
    )

    return history

In [ ]:
def display_learning_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history.history["loss"])
    ax1.plot(history.history["val_loss"])
    ax1.legend(["train", "test"], loc="upper right")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")

    ax2.plot(history.history["acc"])
    ax2.plot(history.history["val_acc"])
    ax2.legend(["train", "test"], loc="upper right")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    plt.show()

## Create Graph Convolutional Layer

In [ ]:
class GraphConvLayer(layers.Layer):
    def __init__(
        self,
        hidden_units,
        dropout_rate=0.2,
        # used to aggregate node messages
        aggregation_type="mean",
        # used to combine node representations with their messages
        combination_type="concat",
        normalize=False,
        *args,
        **kwargs,
    ):
        super(GraphConvLayer, self).__init__(*args, **kwargs)

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
        if self.combination_type == "gated":
            self.update_fn = layers.GRU(
                units=hidden_units,
                activation="tanh",
                recurrent_activation="sigmoid",
                dropout=dropout_rate,
                return_state=True,
                recurrent_dropout=dropout_rate,
            )
        else:
            self.update_fn = create_ffn(hidden_units, dropout_rate)

    def prepare(self, node_repesentations, weights=None):
        # node_repesentations shape is [num_edges, embedding_dim].
        messages = self.ffn_prepare(node_repesentations)
        if weights is not None:
            messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_repesentations):
        # node_indices shape is [num_edges].
        # neighbour_messages shape: [num_edges, representation_dim].
        # node_repesentations shape is [num_nodes, representation_dim]
        num_nodes = node_repesentations.shape[0]
        if self.aggregation_type == "sum":
            aggregated_message = tf.math.unsorted_segment_sum(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "mean":
            aggregated_message = tf.math.unsorted_segment_mean(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "max":
            aggregated_message = tf.math.unsorted_segment_max(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        else:
            raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")

        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        # node_repesentations shape is [num_nodes, representation_dim].
        # aggregated_messages shape is [num_nodes, representation_dim].
        if self.combination_type == "gru":
            # Create a sequence of two elements for the GRU layer.
            h = tf.stack([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "concat":
            # Concatenate the node_repesentations and aggregated_messages.
            h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "add":
            # Add node_repesentations and aggregated_messages.
            h = node_repesentations + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        # Apply the processing function.
        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):
        """Process the inputs to produce the node_embeddings.

        inputs: a tuple of three elements: node_repesentations, edges, edge_weights.
        Returns: node_embeddings of shape [num_nodes, representation_dim].
        """

        node_repesentations, edges, edge_weights = inputs
        # Get node_indices (source) and neighbour_indices (target) from edges.
        node_indices, neighbour_indices = edges[0], edges[1]
        # neighbour_repesentations shape is [num_edges, representation_dim].
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)

        # Prepare the messages of the neighbours.
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)
        # Aggregate the neighbour messages.
        aggregated_messages = self.aggregate(
            node_indices, neighbour_messages, node_repesentations
        )
        # Update the node embedding with the neighbour messages.
        return self.update(node_repesentations, aggregated_messages)

## Create GNN Classifier

In [ ]:
class GNNNodeClassifier(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        num_classes,
        hidden_units,
        aggregation_type="sum",
        combination_type="concat",
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        super(GNNNodeClassifier, self).__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        # Set edge_weights to ones if not provided.
        if self.edge_weights is None:
            self.edge_weights = tf.ones(shape=edges.shape[1])
        # Scale edge_weights to sum to 1.
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        # Create a process layer.
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        # Create the second GraphConv layer.
        self.conv2 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2",
        )
        # Create a postprocess layer.
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        self.compute_logits = layers.Dense(units=num_classes, name="logits")

    def call(self, input_node_indices):
        # Preprocess the node_features to produce node representations.
        print(self.node_features)
        x = self.preprocess(self.node_features)
        # Apply the first graph conv layer.
        x1 = self.conv1((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x1 + x
        # Apply the second graph conv layer.
        x2 = self.conv2((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x2 + x
        # Postprocess node embedding.
        x = self.postprocess(x)
        # Fetch node embeddings for the input node_indices.
        node_embeddings = tf.gather(x, input_node_indices)
        # Compute logits
        return self.compute_logits(node_embeddings)

## Instantiate GNN Classifier

In [ ]:
def trainOnGraphs(graphs):
  x_train, y_train = [], []
  
  for graph in graphs:
    graph_dict = transformGraphForGNN(graph)

    x_train.append(np.array([node_feature for node_feature in graph_dict["info"]["node_features"]]))
    y_train.append(labels.index(graph_dict["label"]))
    # for node_feature in graph_dict["info"]["node_features"]:
    #   x_train.append(node_feature)
    # y_train.append([labels.index(graph_dict["label"])])

  x_train = tf.cast(
      np.array(x_train, dtype=np.float32), 
      dtype=tf.dtypes.float32
  )
  # print(x_train)

  y_train = np.array(y_train, dtype=np.float32)
  # print(y_train)

  # x_train = np.array([1,1,1,1])
  # y_train = np.array([1, 0, 1, 0])

  print(graph_dict["info"].values())

  gnn_model = GNNNodeClassifier(
    graph_info=tuple(graph_dict["info"].values()),
    num_classes=num_classes,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
  )

  # print("GNN output shape:", gnn_model(tf.constant([1, 10])))

  # gnn_model.summary()

   # print("GNN output shape:", gnn_model(np.asarray([1, 10, 100])))

  return run_experiment(gnn_model, x_train, y_train)

import json

HISTORY_PATH = "/content/history.json"

history = trainOnGraphs(graphs).history

# json.dump(history, open(HISTORY_PATH, "w+"))

print(history)

dict_values([array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object), array([[  0,   0,   0,   1,   4,   4,   4,   4,   4,   4,   4,   4,   4,
          4,   8,   8,   8,   9,  17,  17,  17,  17,  17,  17,  17,  17,
         22,  22,  23,  29,  29,  29,  29,  29,  30,  36,  36,  36,  38,
         45,  45,  45,  45,  45,  45,  45,  45,  45,  45,  50,  50,  51,
         57,  57,  57,  58,  61,  61,  61,  61,  61,  61,  63,  63,  63,
         64,  64,  64,  65,  73,  76,  80,  81,  81,  81,  81,  81,  81,
         82,  82,  82,  83,  89,  92,   5,   6,   7,  10,  12,  13,  15,
         16,  18,  19,  21,  24,  26,  27,  28,  31,  32,  33,  37,  39,
         41,  42,  43,  44,  46,  47,  49,  52,  54,  55,  56,  59,  62,
         66,  67,  68,  69,  70,  72,  74,  75,  77,  78,  79,  84,  85,
         86,  88,  90,  91,  93,  57,

TypeError: ignored